### Import Dependencies

In [1]:
import openai
import instructor
from qdrant_client import QdrantClient

from pydantic import BaseModel, Field

### Mock Example

In [2]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name?
"""

In [3]:
openai_response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0
)

print(openai_response.choices[0].message.content)

I am ChatGPT, your AI assistant. How can I help you today?


In [4]:
openai_response

ChatCompletion(id='chatcmpl-D0pO03BWEo2B1pE1mcbN8TSEvND5Y', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am ChatGPT, your AI assistant. How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1769089900, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_2191215734', usage=CompletionUsage(completion_tokens=16, prompt_tokens=28, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Add Instructor (Structured Outputs)

In [5]:
client = instructor.from_openai(openai.OpenAI())

In [6]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [7]:
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)

In [8]:
response

RAGGenerationResponse(answer='I am ChatGPT, an AI language model created by OpenAI. How can I assist you today?')

In [9]:
raw_response

ChatCompletion(id='chatcmpl-D0pO1fFYUIZSnZtn0Qkw8iHYIA009', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_g4RCq53XjiqfrcnpWdGH2evf', function=Function(arguments='{"answer":"I am ChatGPT, an AI language model created by OpenAI. How can I assist you today?"}', name='RAGGenerationResponse'), type='function')]))], created=1769089901, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=25, prompt_tokens=95, total_tokens=120, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [10]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    reasoning: str = Field(description="The reasoning for the answer")

In [11]:
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)

In [12]:
response

RAGGenerationResponse(answer='My name is ChatGPT.', reasoning='I am an AI language model created by OpenAI, and I am commonly known as ChatGPT.')

### RAG Example

In [13]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [14]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "datamodel": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [15]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [16]:
output = rag_pipeline("Can I get a charging cable? Please suggest me a good one.", qdrant_client)

In [17]:
output

{'datamodel': RAGGenerationResponse(answer='Based on the available products, here are some good charging cable options:\n\n1. GREPHONE 2 Pack USB C to Lightning Cable, 6 FT, MFi Certified - Great for fast charging iPhones and iPads with USB-C chargers. Durable with reinforced material and supports PD Fast Charge 3A/30W.\n\n2. iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] - Apple MFi Certified, durable with reinforced joints, supports fast charging 3.0A and data transfer.\n\n3. MUXA 6 Pack Colorful Nylon Lightning Cable - MFi Certified, durable with 12,000+ bend cycles, multiple lengths included.\n\n4. 5 in 1 USB C to Multi Charging Cable 3M/10Ft - Multi-functional cable with Lightning, Type C, and Micro USB connectors, can charge three devices simultaneously.\n\nIf you want a versatile cable for multiple device types, the 5 in 1 USB C Multi Charging Cable is a good choice. For pure iPhone fast charging, the GREPHONE or the Original Apple MFi Certified cables 

In [18]:
print(output["answer"])

Based on the available products, here are some good charging cable options:

1. GREPHONE 2 Pack USB C to Lightning Cable, 6 FT, MFi Certified - Great for fast charging iPhones and iPads with USB-C chargers. Durable with reinforced material and supports PD Fast Charge 3A/30W.

2. iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] - Apple MFi Certified, durable with reinforced joints, supports fast charging 3.0A and data transfer.

3. MUXA 6 Pack Colorful Nylon Lightning Cable - MFi Certified, durable with 12,000+ bend cycles, multiple lengths included.

4. 5 in 1 USB C to Multi Charging Cable 3M/10Ft - Multi-functional cable with Lightning, Type C, and Micro USB connectors, can charge three devices simultaneously.

If you want a versatile cable for multiple device types, the 5 in 1 USB C Multi Charging Cable is a good choice. For pure iPhone fast charging, the GREPHONE or the Original Apple MFi Certified cables are excellent.

Let me know if you want more details o